# Tâche 3 : Question-réponse avec GPT‑2 avec poursuite du pré-entraînement sur un corpus de Sherlock Holmes

**Objectifs**

Évaluer la qualité des réponses d’un modèle de langage **pré‑entraîné** (version **GPT-2 Medium (355M)** sur Hugging Face) lorsqu’on lui pose des questions sur un sujet vu au pré-entraînement.
Dans ce *notebook*, vous poursuivez le pré-entraînement du modèle avec des textes de l'univers de *Sherlock Holmes*.  Comme pour la tâche 2, on doit construire un *prompt* minimal, générer des réponses avec le nouveau modèle, et évaluer la pertinence des résultats. Plusieurs de ces fonctions sont rendues disponibles.

**Objectifs d’apprentissage**
1. Poursuivre le préentraînement d'un modèle pré‑entraîné (Hugging Face) sur un corpus de taille moyenne.
2. Comprendre et expliquer les **limites et apports du pré‑entraînement** sur des textes pertinents au domaine des questions.

Tout comme pour la tâche 2, les **questions** pour évaluer le modèle vous sont fournies. Vous devez comprendre le format des questions chargées en mémoire. La **liste de livres** à utiliser pour le pré-entraînement et la **fonction** pour les monter en mémoire sont également disponibles.

NOTE: Il est important de sauvegarder le modèle pré-entraîné dans cette tâche (ainsi que son tokenizer) car nous les réutilisons pour la tâche 4.

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [1]:
#!pip install datasets
#!pip install accelerate
#!pip install 'transformers[torch]'
#!pip3 install torch torchvision

In [19]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécessite pas un plus grand contexte
model_name = "gpt2-medium"

In [20]:
from datasets import Dataset
from transformers import pipeline, Trainer
import os
import json

import re
import requests

In [21]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

#Mount a google drive folder to save things
if IN_COLAB:
  drive.mount('/content/drive')
  folders_to_mount = ["nlp_tp2_models", "results"]
  for folder in folders_to_mount:
    source = f'/content/drive/MyDrive/uni/nlp/{folder}'
    shortcut = f'/content/{folder}'
    print(f"Mounting {source} to {shortcut}")
    os.symlink(source, shortcut)

repo_url = "https://github.com/XavyShmore/tp2_nlp.git"
if IN_COLAB:
  !git clone {repo_url}
  !cp -r ./tp2_nlp/data .
  pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounting /content/drive/MyDrive/uni/nlp/nlp_tp2_models to /content/nlp_tp2_models


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/uni/nlp/nlp_tp2_models' -> '/content/nlp_tp2_models'

## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier.

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM
def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_path)

    return tokenizer, model

In [23]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")


## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) :
* Construire un prompt à l’aide de la fonction **build_prompt** (rendu disponible).
* Utiliser le modèle (via un pipeline de génération de texte passé en argument) pour générer une réponse à une question.
* Retourner la réponse générée par le modèle.  

Points importants à souligner:
* La fonction *process_entry* doit retourner uniquement la réponse générée par le modèle (sans le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

In [24]:
def build_prompt(entry):
    question = entry.get("question", "")
    prompt = f"Please answer the following question about the Sherlock Holmes Universe with a short answer.\nQuestion: {question}\nAnswer: "
    return prompt

Description des paramètres de génération:
(à compléter...)

In [25]:
def process_entry(entry, prompt_builder, generator):
    prompt = prompt_builder(entry)
    generation_output = generator(
        prompt,
        max_new_tokens=10, # limit the length of the generated answer
        do_sample=False,    # enable sampling for more varied responses
        temperature=0.8,   # control the randomness of the output
        top_k=50,          # consider only the top k most likely next tokens
        num_return_sequences=1
    )
    # The generated_text includes the prompt, so we need to extract only the answer
    generated_text = generation_output[0]['generated_text']
    answer = generated_text[len(prompt):].strip()
    return answer

In [26]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry(entry, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Poursuite du pré-entraînement du modèle GPT-2 (à compléter)

Complétez le code suivant afin de poursuivre le préentraînement de GPT2 sur des textes de *Sherlock Holmes*.

Les étapes à suivre sont de :
* Télécharger le contenu des livres (on rend la fonction disponible)
* Créer un *dataset* (version Hugging Face) d'entraînement à partir de ce contenu
* Tokeniser ce *dataset*
* Faire le pré-entraînement du modèle sur le *dataset* avec la classe ***Trainer*** de Hugging Face
* Faire la sauvegarde du nouveau modèle dans un répertoire (voir *model_path*)

In [27]:
books = {
    "The Sign of the Four": "https://www.gutenberg.org/files/2097/2097-0.txt",
    "The Adventures of Sherlock Holmes": "https://www.gutenberg.org/files/1661/1661-0.txt",
    "The Memoirs of Sherlock Holmes": "https://www.gutenberg.org/files/834/834-0.txt",
    "The Hound of the Baskervilles": "https://www.gutenberg.org/files/2852/2852-0.txt",
    "His Last Bow": "https://www.gutenberg.org/files/2350/2350-0.txt",
    "The Case-Book of Sherlock Holmes": "https://www.gutenberg.org/files/221/221-0.txt"
}

def download_sherlock_dataset(books_to_process):
    data = []

    for title, url in books_to_process.items():
        response = requests.get(url)

        if response.status_code == 200:
            text = response.text

            # Cette expression est plus robuste que ce qui était attendu dans le premier travail pratique
            header_regex = r"(?s)^.*?\*{3}\s*START OF\b.*?\r?\n?\*{3}\s*\r?\n"
            header_pattern = re.compile(header_regex, flags=0)
            clean_text = header_pattern.sub("", text)

            toc_regex = r"(?ims)^\s*contents\s*$.*?^\s*$"
            toc_pattern = re.compile(toc_regex, flags=0)
            clean_text = toc_pattern.sub("", clean_text)

            regex_license_llm = r"(?im)^\s*\*{3} END OF\b.*[\s\S]*\Z"
            license_llm_pattern = re.compile(regex_license_llm, flags=0)
            clean_text = license_llm_pattern.sub("", clean_text)

            # To make it simpler to learn text without learning new lines
            clean_text = re.sub(r"\r\n\r\n", "\n", clean_text)
            clean_text = re.sub(r"\r\n", " ", clean_text)

            #retirer white doubled white char
            clean_text = re.sub(r'\s{2,}', ' ', clean_text)

            data.append(clean_text)
            print(f"Downloaded: {title}")
        else:
            print(f"Failed to download: {title}")

    return "\n".join(data)

Création du *dataset* d'entraînement.

In [28]:
sherlock_text = download_sherlock_dataset(books_to_process=books)

Downloaded: The Sign of the Four
Downloaded: The Adventures of Sherlock Holmes
Downloaded: The Memoirs of Sherlock Holmes
Downloaded: The Hound of the Baskervilles
Downloaded: His Last Bow
Downloaded: The Case-Book of Sherlock Holmes


In [29]:
def create_dataset(text, tokeniser, context_length):
  tokenizer_max_length = tokeniser.model_max_length

  tokeniser.model_max_length = 1000000 # Mentire au tokeniser pour qu'il fonctionne même si on dépasse la limite du modèle
  tokenized_text = tokeniser(sherlock_text)
  tokeniser.model_max_length = tokenizer_max_length # Rétablir la vraie valeur

  text_token_length = len(tokenized_text["input_ids"])
  print(f"Generating {text_token_length} tokens")

  input_ids = [tokenized_text["input_ids"][i*context_length:(i+1)*context_length] for i in range(text_token_length // context_length)]
  attention_mask = [tokenized_text["attention_mask"][i*context_length:(i+1)*context_length] for i in range(text_token_length // context_length)]

  return Dataset.from_dict({
      "input_ids": input_ids,
      "attention_mask": attention_mask
  })



Tokénisation du *dataset* d'entraînement.

In [16]:
tokenizer, model = load_model(model_name)

tokenized_dataset = create_dataset(sherlock_text, tokenizer, max_length)

Generating 639546 tokens


Ajouter dans les cellules suivantes le code dont vous avez besoin pour poursuivre le pré-entraînement du modèle.

In [30]:
model_path = "/nlp_tp2_models/gpt2/gpt2-sherlock-lm"  # Répertoire où sauvegarder le nouveau modèle et le tokenizer

from transformers import TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./nlp_tp2_models/gpt2/gpt2-sherlock-lm/checkpoints",
    overwrite_output_dir=True,

    num_train_epochs=3,

    # 2. BATCH SIZE: Small dataset = requires stability.
    # We use gradient accumulation to simulate a larger batch size (e.g., 32)
    # while keeping memory usage low.
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4, # Effective batch size = 4 * 8 = 32

    # 3. LEARNING RATE: The most critical part.
    # Standard is 5e-5. Since your data is small, sticking to the lower end
    # prevents destroying the pre-trained knowledge.
    learning_rate=5e-5,
    weight_decay=0.01,

    # 4. SCHEDULER: vital for small data
    warmup_steps=50, # Warm up quickly (roughly 1 epoch worth of steps)
    lr_scheduler_type="cosine", # Smooth decay is better than linear for language

    # 5. LOGGING
    logging_steps=10,
    save_steps=50,
    fp16=True, # Use mixed precision if on GPU (much faster)
    report_to="none"
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    )


In [31]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.382900
20,3.180100
30,3.092400
40,3.035200
50,2.956800
60,2.955600
70,2.937600
80,2.883100
90,2.807700
100,2.792600


TrainOutput(global_step=237, training_loss=2.845058924035181, metrics={'train_runtime': 1158.7502, 'train_samples_per_second': 6.467, 'train_steps_per_second': 0.205, 'total_flos': 3479841502396416.0, 'train_loss': 2.845058924035181, 'epoch': 3.0})

Pour conclure cette section, sauvegardez le nouveau modèle et le *tokenizer* afin de les réutiliser dans la tâche 4.

In [32]:
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('/nlp_tp2_models/gpt2/gpt2-sherlock-lm/tokenizer_config.json',
 '/nlp_tp2_models/gpt2/gpt2-sherlock-lm/special_tokens_map.json',
 '/nlp_tp2_models/gpt2/gpt2-sherlock-lm/vocab.json',
 '/nlp_tp2_models/gpt2/gpt2-sherlock-lm/merges.txt',
 '/nlp_tp2_models/gpt2/gpt2-sherlock-lm/added_tokens.json',
 '/nlp_tp2_models/gpt2/gpt2-sherlock-lm/tokenizer.json')

## 4. Génération de réponses avec le nouveau modèle GPT-2

Exécutez la cellule suivante pour générer les réponses aux questions avec le modèle que vous venez de pré-entraîner sur des textes de *Sherlock Holmes*.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail.

In [33]:
questions = "data/questions_sherlock.json"
out_file_name = "pretrained_gpt2_answers.txt"

results = test_on_questions(prompt_builder=build_prompt, model_path=model_path, question_file=questions, out_file_name=out_file_name)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Q0: Where do Sherlock Holmes and Dr. Watson live?
A: In the house of Sir Henry Carey, at
Expected:221B Baker Street, London.
------------------------------------------------------------
Q1: Who is Sherlock Holmes' loyal friend and chronicler?
A: I am not sure.”
�
Expected:Dr. John H. Watson.
------------------------------------------------------------
Q2: Who is considered 'The Woman' by Sherlock Holmes?
A: The woman is a woman who has been married
Expected:Irene Adler.
------------------------------------------------------------
Q3: Which story features the Red-Headed League?
A: The Red-Headed League.
�
Expected:The Adventure of the Red-Headed League.
------------------------------------------------------------
Q4: What is the primary occupation of Sherlock Holmes?
A: A detective.
“What is the
Expected:Consulting detective.
------------------------------------------------------------
Q5: Who is Sherlock Holmes' arch-nemesis?
A: The man who has been the most formidable enemy
Expected:P

## 5. Analyse des résultats

### 5.1 Évaluation quantitative (à compléter)

In [34]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [35]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [36]:
def evaluation_generation(results):
    eval = {"precision":0, "recall":0, "f1":0}
    for i, question, answer, expected_answer in results:
        precision, recall, f1 = evaluate_f1(expected_answer, answer)
        eval["precision"] += precision
        eval["recall"] += recall
        eval["f1"] += f1

    eval["precision"] /= len(results)
    eval["recall"] /= len(results)
    eval["f1"] /= len(results)

    return eval

In [37]:
eval = evaluation_generation(results)
print(eval)

{'precision': 0.10133333333333333, 'recall': 0.1861904761904762, 'f1': 0.12565223665223665}


**Question :** Que pensez-vous de cette évaluation ?

## 5. Analyse qualitative (à faire)

Rédigez **5 à 15 phrases** présentant vos observations et expliquant pourquoi, selon vous, le modèle fournit ce type de réponses.

Vous pouvez ajouter des cellules au besoin.

> Cette étape prépare le terrain pour la tâche 4.